In [1]:
import json
import sqlite3
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

DATABASE_NAME = 'database.db'
TABLE_NAME = 'BAKERIES'

engine = create_engine(f'sqlite:///{DATABASE_NAME}', echo=False)

# Initialize the database
def init_db():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            FirstName TEXT NOT NULL,
            LastName TEXT NOT NULL,
            NID TEXT NOT NULL,
            City TEXT NOT NULL,
            Region TEXT NOT NULL,
            District TEXT NOT NULL,
            Lat REAL NOT NULL,
            Lon REAL NOT NULL,
            HouseholdRisk TEXT NOT NULL,
            BakersRisk TEXT NOT NULL,
            TypeFlour INTEGER NOT NULL,
            TypeBread TEXT NOT NULL,
            BreadRations INTEGER NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

init_db()

df = pd.read_csv("./assets/data/data.csv")
gdf_regions = gpd.read_file('./assets/data/geodatabase/Region.geojson')
gdf_district = gpd.read_file('./assets/data/geodatabase/District.geojson')

# FirstName, LastName, NID, City, Lat, Lon, HouseholdRisk, BakersRisk, TypeFlour, TypeBread, BreadRations
COLs = ['FirstName', 'LastName', 'City', 'HouseholdRisk', 'BakersRisk', 'TypeBread']
df[COLs] = df[COLs].astype(str)
df[COLs] = df[COLs].apply(lambda x: x.str.rstrip())
df[COLs] = df[COLs].apply(lambda x: x.str.lstrip())
df[COLs] = df[COLs].apply(lambda x: x.str.replace(' +', ' '))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ي','ی'))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ئ','ی'))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ك', 'ک'))

COLs = ['NID', 'TypeFlour', 'BreadRations']
df[COLs] = df[COLs].astype(int)

COLs = ['Lat', 'Lon']
df[COLs] = df[COLs].astype(float)

# Drop All NULL Value from Lat & Lon Columns
df.dropna(subset=['Lat', 'Lon'], inplace=True)

# # Remove Duplicate NID
# df.drop_duplicates(subset=['NID'], inplace=True)

# # Remove Same Lat & Lon
# df.drop_duplicates(subset=['Lat', 'Lon'], inplace=True)

# Convert NID to `str`
df['NID'] = df['NID'].apply(lambda x: str(x).zfill(10))

# Reset Index
df.reset_index(drop=True, inplace=True)

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['Lon'], df['Lat'])
)

gdf = gdf.set_crs('EPSG:4326')
gdf_regions = gdf_regions.to_crs('EPSG:4326')
gdf_district = gdf_district.to_crs('EPSG:4326')

gdf_joined_gdf_regions = gpd.sjoin(gdf, gdf_regions, how='left', op='within')
gdf_joined_gdf_district = gpd.sjoin(gdf, gdf_district, how='left', op='within')

df['Region'] = gdf_joined_gdf_regions['Region']
df['District'] = gdf_joined_gdf_district['District']

# Drop All NULL Value from Region & Lon District
df.dropna(subset=['Region', 'District'], inplace=True)

# Save to Database
df.to_sql(name='BAKERIES', con=engine, if_exists='append', index=False)

c:\Users\Pooya\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\Users\Pooya\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


1756

In [5]:
df.columns


Index(['FirstName', 'LastName', 'NID', 'City', 'Lat', 'Lon', 'HouseholdRisk',
       'BakersRisk', 'TypeFlour', 'TypeBread', 'BreadRations', 'Region',
       'District'],
      dtype='object')

In [7]:
# tmp = df[[
#     'City', 'Region', 'District',
#     'HouseholdRisk', 'BakersRisk', 'TypeFlour', 'TypeBread', 'BreadRations'
# ]]

tmp = df[[
    'City', 'Region', 'District', 'TypeBread'
]]

def build_hierarchy(data, keys):
    if not keys:
        return {'size': len(data)}
    result = []
    for key, group in data.groupby(keys[0]):
        
        if key in list(data['TypeBread'].unique()):
            result.append({
                'name': key,
                'size': len(group['TypeBread'])
            })
        else:
            result.append({
                'name': key,
                'children': build_hierarchy(group, keys[1:]) if len(keys) > 1 else len(group['TypeBread'])
            })
            
    return result

# Build the hierarchical structure
# hierarchy = build_hierarchy(tmp, ['City', 'Region', 'District', 'HouseholdRisk', 'BakersRisk', 'TypeFlour', 'TypeBread', 'BreadRations'])
hierarchy = build_hierarchy(tmp, ['City', 'Region', 'District', 'TypeBread'])

# Convert to JSON
hierarchical_json = {'name': 'Root', 'children': hierarchy}

hierarchical_json

{'name': 'Root',
 'children': [{'name': 'مشهد',
   'children': [{'name': 'منطقه 1',
     'children': [{'name': 'ناحیه 1',
       'children': [{'name': 'بربری', 'size': 4},
        {'name': 'تافتون', 'size': 6},
        {'name': 'سنگک', 'size': 5},
        {'name': 'لواش', 'size': 4}]},
      {'name': 'ناحیه 2',
       'children': [{'name': 'بربری', 'size': 9},
        {'name': 'تافتون', 'size': 12},
        {'name': 'سنگک', 'size': 9},
        {'name': 'لواش', 'size': 11}]}]},
    {'name': 'منطقه 10',
     'children': [{'name': 'ناحیه 1',
       'children': [{'name': 'بربری', 'size': 8},
        {'name': 'تافتون', 'size': 9},
        {'name': 'سنگک', 'size': 9},
        {'name': 'لواش', 'size': 5}]},
      {'name': 'ناحیه 2',
       'children': [{'name': 'بربری', 'size': 15},
        {'name': 'تافتون', 'size': 15},
        {'name': 'سنگک', 'size': 18},
        {'name': 'لواش', 'size': 13}]},
      {'name': 'ناحیه 3',
       'children': [{'name': 'بربری', 'size': 24},
        {'name': '

In [11]:
with open('sunburst_data.json', 'w') as json_file:
    json.dump(hierarchical_json, json_file, indent=4)